In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [18]:
def feature_eng(train):
    # train = df_in.copy()
    #Replacing NA values in Bed Grade Column for both Train and Test datssets
    train['Bed Grade'].fillna(train['Bed Grade'].mode()[0], inplace = True)

    #Replacing NA values in  Column for both Train and Test datssets
    train['City_Code_Patient'].fillna(train['City_Code_Patient'].mode()[0], inplace = True)

    # Label Encoding Stay column in train dataset
    le = LabelEncoder()
    # train["Stay"] = le.fit_transform(train["Stay"].astype('str'))

    #Label Encoding all the columns in Train and test datasets
    for i in ['Hospital_type_code', 'Hospital_region_code', 'Department',
            'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']:
        le = LabelEncoder()
        train[i] = le.fit_transform(train[i].astype(str))

    return train

In [19]:
df_los = pd.read_csv("los.csv")

In [20]:
feature_eng(df_los).head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,318439,21,2,3,2,3,2,3,0,2.0,17006,2.0,0,2,2,7,3095.0
1,318440,29,0,4,0,2,2,3,5,2.0,17006,2.0,1,2,4,7,4018.0
2,318441,26,1,2,1,3,2,1,3,4.0,17006,2.0,0,2,3,7,4492.0
3,318442,6,0,6,0,3,2,1,5,2.0,17006,2.0,1,2,3,7,4173.0
4,318443,28,1,11,0,2,2,2,5,2.0,17006,2.0,1,2,4,7,4161.0


In [ ]:
def get_countid_enocde(train, cols, name):
    temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
    train = pd.merge(train, temp, how='left', on= cols)
    train[name] = train[name].astype('float')
    train[name].fillna(np.median(temp[name]), inplace = True)
    return train

def test_train_split(df):  
    #Spearating Train and Test Datasets
    train = df[df['Stay']!=-1]
    test = df[df['Stay']==-1]

    train = get_countid_enocde(train, ['patientid'], name = 'count_id_patient')
    train = get_countid_enocde(train, 
                                    ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
    train = get_countid_enocde(train, 
                                    ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')

    # Droping duplicate columns
    test1 = test.drop(['Stay', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)
    train1 = train.drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)

    # Splitting train data for Naive Bayes and XGBoost
    X1 = train1.drop('Stay', axis =1)
    y1 = train1['Stay']
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)

    return X_train, X_test, y_train, y_test